In [3]:
from langchain_groq import ChatGroq

In [4]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key="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"
)
response=llm.invoke("The first person to land on moon")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [5]:
pip install  langchain_community beautifulsoup4

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-36731")
page_data=loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Retail Supervisor - Simpsonville, KY

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsRetail StoresRetail Supervisor - Simpsonville, KYSimpsonville, KentuckyBecome a Part of th

In [7]:
from langchain_core.prompts import PromptTemplate
prompt_extract=PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ###INSTRUCTION:
    The scraped text is from the career's page of some company's website.
    Your job is to extract the job postings and return them in JSON format containing the following keys:'role','experience','skill' and 'description'.
    Only return the valid JSON
    ### VALID JSON (NO PREAMBLE SPECIFICALLY):
    """
)
# create a pipe with the llm object that have our api 
chain_extract=prompt_extract|llm
res=chain_extract.invoke(input={'page_data':page_data}) #page data we created in the webbaseloader
print(res.content)

```json
{
  "role": "Retail Supervisor",
  "experience": "1 year of retail and/or consumer service experience",
  "skill": [
    "Passion for engaging with consumers and encouraging them to become Nike Members",
    "Digital literacy—desire to learn and adapt to technology needs with agility",
    "Ability to communicate in English",
    "Physical requirements include the ability to twist, bend, squat, reach, climb a ladder, and stand for extended periods of time with or without reasonable accommodation"
  ],
  "description": "The mission of a Nike store Specialist is clear: deliver a premium in-store experience to our consumers. Along with balancing an associate's responsibilities, you're able to expertly handle complex situations with autonomy and poise. In addition to shared leadership responsibilities, you will focus on one of five Areas of Expertise where you'll gain critical experiences that help propel you along your development journey."
}
```


In [8]:
type(res.content)

str

In [9]:
from langchain_core.output_parsers import JsonOutputParser
json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'Retail Supervisor',
 'experience': '1 year of retail and/or consumer service experience',
 'skill': ['Passion for engaging with consumers and encouraging them to become Nike Members',
  'Digital literacy—desire to learn and adapt to technology needs with agility',
  'Ability to communicate in English',
  'Physical requirements include the ability to twist, bend, squat, reach, climb a ladder, and stand for extended periods of time with or without reasonable accommodation'],
 'description': "The mission of a Nike store Specialist is clear: deliver a premium in-store experience to our consumers. Along with balancing an associate's responsibilities, you're able to expertly handle complex situations with autonomy and poise. In addition to shared leadership responsibilities, you will focus on one of five Areas of Expertise where you'll gain critical experiences that help propel you along your development journey."}

In [10]:
type(json_res)

dict

In [11]:
pip install pandas

In [12]:
import pandas as pd
df=pd.read_csv("my_portfolio.csv")

In [13]:
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [17]:
import chromadb
import uuid
client=chromadb.PersistentClient('vectorstore') #when we use client is creates a db over memory but if we use persistent client is creates it over disk
collection=client.get_or_create_collection(name="portfolio")
if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links":row["Links"]},
                      ids=[str(uuid.uuid4())])

In [23]:
links=collection.query(query_texts=["Experience in python","Experience in  React"],n_results=2).get('metadatas',[])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [27]:
link=collection.query(query_texts=job['skill'],n_results=2).get('metadatas',[])
link

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}]]

In [29]:
job=json_res
job['skill']

['Passion for engaging with consumers and encouraging them to become Nike Members',
 'Digital literacy—desire to learn and adapt to technology needs with agility',
 'Ability to communicate in English',
 'Physical requirements include the ability to twist, bend, squat, reach, climb a ladder, and stand for extended periods of time with or without reasonable accommodation']

In [33]:
job

{'role': 'Retail Supervisor',
 'experience': '1 year of retail and/or consumer service experience',
 'skill': ['Passion for engaging with consumers and encouraging them to become Nike Members',
  'Digital literacy—desire to learn and adapt to technology needs with agility',
  'Ability to communicate in English',
  'Physical requirements include the ability to twist, bend, squat, reach, climb a ladder, and stand for extended periods of time with or without reasonable accommodation'],
 'description': "The mission of a Nike store Specialist is clear: deliver a premium in-store experience to our consumers. Along with balancing an associate's responsibilities, you're able to expertly handle complex situations with autonomy and poise. In addition to shared leadership responsibilities, you will focus on one of five Areas of Expertise where you'll gain critical experiences that help propel you along your development journey."}

In [34]:
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [32]:
prompt_email = PromptTemplate.from_template( 
    """ 
    ### JOB DESCRIPTION: 
    {job_description} 
    ### INSTRUCTION: You are Snehashis, a business development executive at XYZ. 
    XYZ is an AI & Software Consulting company dedicated to facilitating the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. 
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of XYZ in fulfilling their needs. 
    Also add the most relevant ones from the following links to showcase XYZ's portfolio: {link_list}
    Remember you are Snehashis, BDE at XYZ. 
    Do not provide a preamble. 
    ### EMAIL (NO PREAMBLE SPECIFICALLY): 
    """ ) 
chain_email = prompt_email | llm 
res = chain_email.invoke({"job_description": str(job), "link_list": links}) 
print(res.content)

Subject: Enhance Your Retail Experience with AI-Powered Solutions from XYZ

Dear Hiring Manager,

I came across the Retail Supervisor role at Nike and was impressed by the emphasis on delivering a premium in-store experience to consumers. As a Business Development Executive at XYZ, I'd like to introduce you to our AI and software consulting services that can help elevate your retail operations and enhance customer engagement.

Our team at XYZ has extensive experience in developing tailored solutions that cater to the unique needs of retail businesses. We've empowered numerous enterprises with automated tools that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise in machine learning and AI can help you streamline store operations, improve inventory management, and provide personalized customer experiences.

I'd like to highlight a few examples from our portfolio that demonstrate our capabilities in retail technology:

* Our machin